Item 42 Prefer Public Attributes Over Private Ones

Things to Remember
- Private attributes aren't rigorously enforced by the Python compiler.
- Plan from the beginning to allow subclasses to do more with your internal APIs and attributes instead of choosing to lock them out.
- Use documentation of protected fields to guide subclasses instead of trying to force access control with private attributes.
- Only consider using private attributes to avoid naming conflicts with subclasses that are out of your control.

In [ ]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        # prefixing an attribute's name
        # with a double underscore to 
        # make it private
        self.__private_field = 10 
    def get_private_field(self):
        return self.__private_field

In [ ]:
foo = MyObject()
# - use dot operator to access
#   public attributes 
assert foo.public_field == 5
assert foo.get_private_field() == 10

In [ ]:
# - you can't access private attributes
#   from outside the classes
foo.__private_field # error

In [ ]:
class MyOtherObject:
    def __init__(self):
        self.__private_field = 71
    @classmethod
    # - class methods have access to private attributes
    #   because they are declared within the surrounding
    #   class block
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field


In [ ]:
# class methods have access to private attributes
bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

In [ ]:
# a subclass can't access its parent calss's private fields
class MyParentObject:
    def __init__(self):
        self.__private_field = 71

class MyChildObject(MyParentObject):
    def get_private_field(self):
        # - will cause error as you can't 
        #   access parent class's private
        #   fields from child a subclass 
        return self.__private_field

In [ ]:
baz = MyChildObject()
baz.get_private_field() # error

How to access the private attributes without asking for permission
- the private attribute in MyParentObject, \__private\_field, will be translated to \_MyParentObject\__private\_field
- so if you know the transformation of the attribute name, you can access the private attributes directly  
 

In [ ]:
# access the private attribute directly
assert baz._MyParentObject__private_field == 71

In [ ]:
#  the real name of the private fields
print(baz.__dict__) # _MyParentObject__private_field 

Why Python doesn't enforce strict visibility 
- Python programmers believe the benefits of being open and should permit unplanned extension of classes by default
- Attribute access methods like \__getattr\__ (see Item 47) enables you to mess around with the internals of objects whenever you wish anyway.
- follow naming convention to prefix a field with a single underscore to indicate it's protected 

In [ ]:
# wrong approach
class MyStringClass:
    def __init__(self, value):
        self.__value = value
    def get_value(self):
         # - only return string,
         #   which makes it really
         #   not flexible  
        return str(self.__value)

In [ ]:
foo = MyStringClass(5)
assert foo.get_value() == '5'

- By choosing private attributes in your approach, you're only making subclass overrides and extensions cumbersome and brittle  

In [ ]:
# I want to return an integer instead
class MyIntegerSubclass(MyStringClass):
    # - override this method to work around
    #   the deficiency that the parent's
    #   method only returns a string
    def get_value(self):
        # - this is brittle and can break easily 
        return int(self._MyStringClass__value)

In [ ]:
foo = MyIntegerSubclass('5')
assert foo.get_value() == 5

In [ ]:
# - class hierachy changes and the private attribute 
#   references are no longer valid
class MyBaseClass:
    def __init__(self, value):
        self.__value = value
    def get_value(self):
        return self.__value

class MyStringClass(MyBaseClass):
    def get_value(self):
        return str(super.get_value()) # updated

class MyIntegerSubclass(MyStringClass):
    def get_value(self):
        # the reference is no longer valid
        return int(self._MyStringClass__value)

In [ ]:
foo = MyIntegerSubclass(5)
foo.get_value() # error

- In general it's better to err on the side of allowing subclasses to do more by using protected attributes.
- Document each protected field and explain which fields are internal APIs available to subclasses and which should be left alone entirely

When to use private attributes?
- Avoid naming conflicts with subclasses which might be out of your control
- Such conflicts are especially possible with attribute names that are very common

In [ ]:
class ApiClass:
    def __init__(self):
        self._value = 5 # protected field
    def get(self):
        return self._value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello' # conflicts

In [ ]:
a = Child()
print(f'{a.get()} and {a._value} should be different')

In [ ]:
# use private attributes
class ApiClass:
    def __init__(self):
        self.__value = 5 # private field
    def get(self):
        return self.__value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello' # ok

In [ ]:
a = Child()
print(f'{a.get()} and {a._value} should be different')